<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/DistillationNRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [3]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox
    from che import Props

import numpy as np
from scipy.optimize import root

In [4]:
p=Props(['Ethanol','Water'])

In [5]:
d=DotDict()

d.P = 1e5
d.N = 5
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.2,0.8])
d.FT =320.
d.D = 0.25*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10.


In [6]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [7]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_minmax = np.min(d.bp), np.max(d.bp)

# Bug in root solver: convergence failure if T_guess is exactly linearly spaced
d.T = RangeArray(np.round(np.linspace(*bp_scaled,d.N)), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [8]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [15]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB[d.F_stage-1] += p.Hl(d.F*d.Fz, d.FT)
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [16]:
eqs_wrapped = wrap(eqs)

In [17]:
eqs(d2)

(array([-1.30967237e-10, -1.69893610e-09, -6.18456397e-10, -2.18278728e-11,
         5.02041075e-10, -1.67347025e-10, -2.32103048e-09,  1.87719706e-09,
        -1.45519152e-10,  8.00355338e-10,  3.97903932e-13, -3.26849658e-13,
        -1.42108547e-13,  1.42108547e-13, -9.09494702e-13,  9.23705556e-13,
        -7.38964445e-13,  7.67386155e-13,  9.52127266e-13, -1.02318154e-12,
        -1.19581819e-05, -1.11758709e-08, -1.49011612e-08, -4.47034836e-08,
         3.91155481e-08, -1.57247996e-06,  0.00000000e+00,  1.31167471e-05]),
 {'CONDENSER': array([1.31167471e-05]),
  'EB': array([-1.19581819e-05, -1.11758709e-08, -1.49011612e-08, -4.47034836e-08,
          3.91155481e-08]),
  'EQUIL': array([[-1.30967237e-10, -1.69893610e-09],
         [-6.18456397e-10, -2.18278728e-11],
         [ 5.02041075e-10, -1.67347025e-10],
         [-2.32103048e-09,  1.87719706e-09],
         [-1.45519152e-10,  8.00355338e-10]]),
  'Lin': array([[180.53030829,  69.46969171],
         [160.50328059,  88.24737

In [18]:
sol=root(eqs_wrapped, x, )
print(sol)
solx=sol.x

    fjac: array([[-6.48648808e-02, -2.52666829e-02, -2.33987147e-10,
         4.64455290e-11, -1.87821505e-09, -6.86701639e-10,
         2.82275411e-09, -1.92202913e-09, -4.80235782e-11,
        -2.04968149e-09, -4.06329246e-12,  4.22086138e-12,
         5.00297418e-13, -8.60564632e-13,  2.11698259e-12,
        -2.11151215e-12,  5.76906634e-13, -6.01751689e-13,
        -4.03315739e-13,  6.24723907e-13,  6.73897706e-01,
        -6.61161128e-01,  3.37720971e-08,  4.81942086e-08,
        -7.64552857e-08, -2.77415833e-05, -2.69323136e-17,
        -3.22307284e-01],
       [-3.41777242e-02, -1.33131676e-02, -5.32792015e-02,
        -2.89102804e-02, -3.53178449e-09, -1.29576996e-09,
         5.42570981e-09, -3.23140578e-09, -3.19874552e-10,
        -3.48127229e-09, -4.75342450e-12,  5.01271468e-12,
         2.27055855e-14, -6.66396330e-13,  3.56247698e-12,
        -3.54243157e-12,  2.07092028e-12, -2.11205793e-12,
        -1.71326152e-12,  2.11796644e-12,  3.55002362e-01,
         4.48448659e

In [19]:
res=xtod(sol.x,d2)
print(res.Lx)
print(res.Vy)

[[0.64523761 0.35476239]
 [0.49593656 0.50406344]
 [0.41502449 0.58497551]
 [0.17359086 0.82640914]
 [0.02595959 0.97404041]]
[[0.72212123 0.27787877]
 [0.65225893 0.34774107]
 [0.62144007 0.37855993]
 [0.51955532 0.48044468]
 [0.21385867 0.78614133]]


In [20]:
eqs(xtod(sol.x,d2))

(array([-1.30967237e-10, -1.69893610e-09, -6.18456397e-10, -2.18278728e-11,
         5.02041075e-10, -1.67347025e-10, -2.32103048e-09,  1.87719706e-09,
        -1.45519152e-10,  8.00355338e-10,  3.97903932e-13, -3.26849658e-13,
        -1.42108547e-13,  1.42108547e-13, -9.09494702e-13,  9.23705556e-13,
        -7.38964445e-13,  7.67386155e-13,  9.52127266e-13, -1.02318154e-12,
        -1.19581819e-05, -1.11758709e-08, -1.49011612e-08, -4.47034836e-08,
         3.91155481e-08, -1.57247996e-06,  0.00000000e+00,  1.31167471e-05]),
 {'CONDENSER': array([1.31167471e-05]),
  'EB': array([-1.19581819e-05, -1.11758709e-08, -1.49011612e-08, -4.47034836e-08,
          3.91155481e-08]),
  'EQUIL': array([[-1.30967237e-10, -1.69893610e-09],
         [-6.18456397e-10, -2.18278728e-11],
         [ 5.02041075e-10, -1.67347025e-10],
         [-2.32103048e-09,  1.87719706e-09],
         [-1.45519152e-10,  8.00355338e-10]]),
  'Lin': array([[180.53030829,  69.46969171],
         [160.50328059,  88.24737